# Balance Ops
Balance implementation is buit on top of four simple concepts:
- Hold - hold amount for order execution
- Release - release the amount held
- Withdraw - withdraw an asset (trading fees, orders execution)
- Deposit - to deposit gain

Also, there's Release With Gain. This one is specific to Futures Balance. Since on contract purchase your money won't be withdrawn but held instead, Release With Gain is used for both 1) releasing the collateral for contracts purchased and 2) deposit the gain (or withdraw loss).  
Here's the sample to show an order flow in terms of the balance

**NOTE**: 
- This notebook covers implementation details, you don't need to hold/release/withdraw/deposit anything directly from within a trading strategy. It is done by FuturesBroker automatically.
- For details regarding Short position please see the 'short' notebook

## Order flow within Long position in terms of balance operations

In [1]:
from decimal import Decimal
from backintime.broker.futures.balance import FuturesBalance

balance = FuturesBalance(Decimal(10000))    # Initialize our balance with 10k
fee = Decimal(10)    # Suppose our trading fee is 10

# Suppose there is an order for BUY posted
balance.hold_usd(Decimal(5000) + fee)   # Hold USD for order execution (with fees)

# Suppose the BUY order is executed and we purchased some contracts
balance.withdraw_usd(fee)    # Withdraw trading fees 
balance.deposit_contracts(Decimal(1))  # Deposit purchased contracts

# Suppose there is an order for SELL posted
balance.hold_contracts(Decimal(1))   # Hold contracts for order execution

# Suppose the SELL order is executed and we now sell contracts with the gain of 100
gain = Decimal(100)   # Suppose our gain is 100
net_gain = gain - fee  # We're going to use net gain further
balance.withdraw_contracts(Decimal(1))  # Withdraw sold contracts
balance.release_with_gain(collateral=Decimal(5000), gain=net_gain)  # Release collateral and deposit gain

print(balance)

Balance(usd_balance=10080.00, available_usd_balance=10080.00, contracts=0, available_contracts=0)


## FuturesBalance vs FuturesBalanceManager

Although `FuturesBalance` provides a set of simple methods to manipulate balance, using it during orders flow tend to be quite verbose. `FuturesBalanceManager` is used instead since Apr. 16 build. This class introduces another layer of abstraction and basically used as a bridge between low-level balance operations and the concepts of orders: submission, execution, cancellation. Key differencies between these two:

FuturesBalance:
- Low level operations
- Operates with decimals (`decimal.Decimal`)
- Modifies the balance directly
- Used by FuturesBalanceManager

FuturesBalanceManager:
- Higher level abstraction
- Supports shared positions for TP/SL orders
- Operates with orders, amounts and fill prices
- Knows how to modify the balance in accordance with order submission/execution/cancellation
- Relies on FuturesBalance's low-level methods to actually modify the balance
- Used by FuturesBroker

With `FuturesBalanceManager` the example above can be rewritten as follows:

In [2]:
from decimal import Decimal
from backintime.broker.base import OrderSide, MarketOrderOptions, TakeProfitOptions, StopLossOptions
from backintime.broker.futures.balance import FuturesBalanceManager

balance_manager = FuturesBalanceManager(Decimal('10000'), 
                                        per_contract_init_margin=Decimal('5000'),
                                        per_contract_fee=Decimal('10'),
                                        contract_quotient=Decimal('1'))

buy_options = MarketOrderOptions(OrderSide.BUY, amount=Decimal('5000'))
sell_options = MarketOrderOptions(OrderSide.SELL, amount=Decimal('1'))

balance_manager.hold_funds(buy_options)
balance_manager.buy(amount=Decimal('5000'), fill_price=Decimal('100'))

balance_manager.hold_funds(sell_options)
balance_manager.sell(amount=Decimal('1'), fill_price=Decimal('200'))

print(balance_manager.balance)

SELL, gain: 90
BalanceInfo(usd_balance=10080.00, available_usd_balance=10080.00, contracts=0, available_contracts=0)


## Shared positions for TP/SL orders
Shared position is implemented in `SharedPositionManager` class (`src/backintime/broker/futures/balance.py`) and `FuturesBalanceManager` uses it. This facilitates multiple orders being posted for a greater amount in sum than there's available on the balance, but the amount of each order must fit the available balance (less than or equal). If any of the orders get executed, others should simply be cancelled (and it is done automatically).   
Here's the snippet to ilustrate the idea:

In [3]:
from decimal import Decimal
from backintime.broker.futures.balance import FuturesBalance, SharedPositionManager

balance = FuturesBalance(Decimal('0'))
balance.deposit_contracts(Decimal('10'))   # just so we would have something
print(f"Deposit contracts: {balance}")

position = SharedPositionManager(balance)

print(f"Hold 10 and actually acquired: {position.hold_shared_contracts(Decimal('10'))}")
print(f"Again hold 10 and actually acquired: {position.hold_shared_contracts(Decimal('10'))}")

position.release_shared_contracts(Decimal('10'))
print(f"Release 10 and the balance now: {balance}")
position.release_shared_contracts(Decimal('10'))
print(f"Again release 10 and the balance now: {balance}")

balance

Deposit contracts: Balance(usd_balance=0.00, available_usd_balance=0.00, contracts=10, available_contracts=10)
Hold 10 and actually acquired: 10
Again hold 10 and actually acquired: 0
Release 10 and the balance now: Balance(usd_balance=0.00, available_usd_balance=0.00, contracts=10, available_contracts=0)
Again release 10 and the balance now: Balance(usd_balance=0.00, available_usd_balance=0.00, contracts=10, available_contracts=10)


Balance(usd_balance=0.00, available_usd_balance=0.00, contracts=10, available_contracts=10)

The code above does not do much, indeed. It acquires some amount of contracts for shared position, and then releases it. Let's see how actual TP/SL execution might look like. This time I'm going to use FuturesBalanceManager here

In [4]:
from decimal import Decimal
from backintime.broker.base import OrderSide, LimitOrderOptions, TakeProfitOptions, StopLossOptions
from backintime.broker.futures.balance import FuturesBalanceManager

balance_manager = FuturesBalanceManager(Decimal('10000'), 
                                        per_contract_init_margin=Decimal('5000'),
                                        per_contract_fee=Decimal('10'),
                                        contract_quotient=Decimal('1'))

# Enter Long
balance_manager.hold_funds(LimitOrderOptions(order_side=OrderSide.BUY, order_price=Decimal(100), amount=Decimal('5000')))
balance_manager.buy(Decimal('5000'), Decimal(100))

# Hold position for TP/SL
balance_manager.hold_position(OrderSide.SELL, TakeProfitOptions(amount=Decimal('1'), trigger_price=200))
balance_manager.hold_position(OrderSide.SELL, StopLossOptions(amount=Decimal('1'), trigger_price=50))

# Close Long at TP level
balance_manager.sell(amount=Decimal('1'), fill_price=Decimal(200))
balance_manager.balance

SELL, gain: 90


BalanceInfo(usd_balance=10080.00, available_usd_balance=10080.00, contracts=0, available_contracts=0)